In [1]:
import pandas as pd
import numpy as np

train_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/train.csv')
train_demo = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv')
test_df = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/test.csv')
test_demo = pd.read_csv('/kaggle/input/cmi-detect-behavior-with-sensor-data/test_demographics.csv')

In [2]:
train_demo.rename(columns={'Sequence_ID': 'sequence_id'}, inplace=True)
test_demo.rename(columns={'Sequence_ID': 'sequence_id'}, inplace=True)

In [3]:
sensor_cols = [col for col in train_df.columns if any(s in col for s in ['acc_', 'gyro_', 'thermopile_', 'tof_'])]

In [4]:
def preprocess(df, demo_df, sensor_cols, label_col='behavior', max_len=300):
    X_seq, X_demo, y = [], [], []

    # Fix column name if needed
    if 'sequence_id' not in demo_df.columns:
        demo_df.rename(columns={demo_df.columns[0]: 'sequence_id'}, inplace=True)

    for seq_id, group in df.groupby("sequence_id"):
        group = group.sort_values("sequence_counter")

        # Replace deprecated fillna
        sensor_data = group[sensor_cols].ffill().bfill().values

        # Pad/trim
        if sensor_data.shape[0] > max_len:
            sensor_data = sensor_data[:max_len]
        else:
            pad = np.zeros((max_len - sensor_data.shape[0], len(sensor_cols)))
            sensor_data = np.vstack([sensor_data, pad])

        demo_row = demo_df[demo_df['sequence_id'] == seq_id].drop(columns=['sequence_id'], errors='ignore')
        demo_vals = demo_row.values.squeeze() if not demo_row.empty else np.zeros(demo_df.shape[1] - 1)

        X_seq.append(sensor_data)
        X_demo.append(demo_vals)

        if label_col in group:
            y.append(group[label_col].iloc[0])

    return (
        np.array(X_seq, dtype=np.float32),
        np.array(X_demo, dtype=np.float32),
        np.array(y) if y else None
    )

In [5]:
X_train_seq, X_train_demo, y_train = preprocess(train_df, train_demo, sensor_cols)
X_test_seq, X_test_demo, _ = preprocess(test_df, test_demo, sensor_cols)

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)  # string → int

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class SensorDataset(Dataset):
    def __init__(self, X_seq, X_demo, y=None):
        self.X_seq = torch.tensor(X_seq, dtype=torch.float32)
        self.X_demo = torch.tensor(X_demo, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long) if y is not None else None

    def __len__(self):
        return len(self.X_seq)

    def __getitem__(self, idx):
        if self.y is not None:
            return self.X_seq[idx], self.X_demo[idx], self.y[idx]
        return self.X_seq[idx], self.X_demo[idx]

train_dataset = SensorDataset(X_train_seq, X_train_demo, y_train_encoded)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [8]:
import torch.nn as nn

class CNNGRUWithDemo(nn.Module):
    def __init__(self, input_dim, demo_dim, hidden_dim=128, num_classes=2):
        super().__init__()
        self.cnn = nn.Sequential(
            nn.Conv1d(input_dim, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )
        self.gru = nn.GRU(64, hidden_dim, batch_first=True, bidirectional=True)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim * 2 + demo_dim, 64),
            nn.ReLU(),
            nn.Linear(64, num_classes)
        )

    def forward(self, x_seq, x_demo):
        x_seq = x_seq.permute(0, 2, 1)        # (B, F, T)
        x = self.cnn(x_seq)                   # (B, C, T//2)
        x = x.permute(0, 2, 1)                # (B, T//2, C)
        _, h = self.gru(x)                    # h: (2, B, H)
        h = torch.cat((h[-2], h[-1]), dim=1)  # bidirectional concat

        x_all = torch.cat((h, x_demo), dim=1)
        return self.classifier(x_all)

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNNGRUWithDemo(input_dim=X_train_seq.shape[2], demo_dim=X_train_demo.shape[1]).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(10):
    model.train()
    total_loss = 0
    for x_seq, x_demo, y in train_loader:
        x_seq, x_demo, y = x_seq.to(device), x_demo.to(device), y.to(device)
        optimizer.zero_grad()
        output = model(x_seq, x_demo)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss:.4f}")

Epoch 1: Loss = nan
Epoch 2: Loss = nan
Epoch 3: Loss = nan
Epoch 4: Loss = nan
Epoch 5: Loss = nan
Epoch 6: Loss = nan
Epoch 7: Loss = nan
Epoch 8: Loss = nan
Epoch 9: Loss = nan
Epoch 10: Loss = nan


In [10]:
test_dataset = SensorDataset(X_test_seq, X_test_demo)
test_loader = DataLoader(test_dataset, batch_size=64)

model.eval()
preds = []
with torch.no_grad():
    for x_seq, x_demo in test_loader:
        x_seq, x_demo = x_seq.to(device), x_demo.to(device)
        output = model(x_seq, x_demo)
        pred = torch.argmax(output, dim=1).cpu().numpy()
        preds.extend(pred)

In [11]:
submission_df = test_demo[['sequence_id']].copy()
submission_df['behavior'] = preds
submission_df.to_parquet("submission.parquet", index=False)